In [1]:
import tensorflow as tf
import os

In [2]:
current_directory=os.getcwd()

In [3]:
train_path=os.path.join(current_directory,"dataset","Train")
test_path=os.path.join(current_directory,"dataset","Test")

In [4]:
valid_path=os.path.join(current_directory,"dataset","Valid")

In [5]:
training_set=tf.keras.utils.image_dataset_from_directory(
    train_path,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    image_size=(128,128),
    shuffle=True,
    interpolation="bilinear")

Found 900 files belonging to 3 classes.


In [6]:
validation_set=tf.keras.utils.image_dataset_from_directory(
    valid_path,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    image_size=(128,128),
    shuffle=True,
    interpolation="bilinear")

Found 300 files belonging to 3 classes.


In [7]:
labels=training_set.class_names
labels

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [9]:
cnn=tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation="relu",input_shape=[128,128,3]))
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Dropout(0.25))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=1500,activation="relu"))
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Dense(units=3,activation="softmax"))

c:\bootcamp\Potato_leaf_disease_prediction\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'dataset',  # Replace with your dataset folder
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [19]:
training_history=cnn.fit(training_set,validation_data=validation_set,epochs=4
                         )

Epoch 1/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9813 - loss: 0.0896 - val_accuracy: 0.9667 - val_loss: 0.1360
Epoch 2/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.9902 - loss: 0.0365 - val_accuracy: 0.8967 - val_loss: 0.3014
Epoch 3/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9699 - loss: 0.0696 - val_accuracy: 0.9567 - val_loss: 0.1874
Epoch 4/4
29/29 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.9836 - loss: 0.0307 - val_accuracy: 0.9500 - val_loss: 0.1596


In [20]:
train_loss,train_accuracy=cnn.evaluate(training_set)
print('train_accuaracy',train_accuracy)

29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 622ms/step - accuracy: 1.0000 - loss: 0.0082
train_accuaracy 1.0


In [21]:
val_loss,val_accuracy=cnn.evaluate(validation_set)
print('validation accuracy',val_accuracy)

10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 544ms/step - accuracy: 0.9504 - loss: 0.1211
validation accuracy 0.949999988079071


In [22]:
cnn.save("trained_plant_disease_model.keras")